# Multiagent Pattern - Multiagent Collaboration

<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together. 

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.


In [1]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%%python3` not found.


In [2]:
import sys
print("Python version:", sys.version)

Python version: 3.10.14 (main, Mar 19 2024, 21:46:16) [Clang 15.0.0 (clang-1500.1.0.2.5)]


In [3]:
%pip install openai python-dotenv graphviz colorama

Note: you may need to restart the kernel to use updated packages.


In [4]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [5]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)

Updated sys.path: ['/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/usr/local/Cellar/python@3.10/3.10.14/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Users/glin/Documents/GitHub/M-LLAP/venv2/lib/python3.10/site-packages', '/Users/glin/Documents/GitHub/SagaLLM/applications/complex_tasks/src']


## The Agent Class

First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.

In [11]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


API Key set: True


In [12]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")

ModuleNotFoundError: No module named 'src'

Let's create some example agent, to see how it works.

In [11]:
agent_example = Agent(
    name="Poet Agent",
    backstory="You are a well-known poet, who enjoys creating high quality poetry.",
    task_description="Write a poem about the meaning of life",
    task_expected_output="Just output the poem, without any title or introductory sentences",
)

In [13]:
print(agent_example.run())

In the cradle of dawn, where whispers begin,  
Life unfurls its tapestry, woven thin.  
A dance of moments, fleeting and bright,  
In shadows and sun, both day and night.  

The meaning hides in the gentle breeze,  
In the rustle of leaves, the hum of bees.  
In laughter shared and tears that fall,  
In the quiet echo of a distant call.  

It's in the eyes of a child, wide with wonder,  
In the storm's roar and the softest thunder.  
In every heartbeat, a silent prayer,  
In the bonds we forge, the love we share.  

Life's meaning, elusive, like a dream,  
Flows in the river, a ceaseless stream.  
In the quest for truth, in the search for grace,  
In the endless journey, we find our place.  

So let us cherish each fleeting breath,  
In the dance of life, defying death.  
For in every moment, both joy and strife,  
We find the essence, the meaning of life.  


You can also associate tools with the agent. Let's create a tool for writing some string into a CSV.

In [14]:
from src.tool_agent.tool import tool

In [15]:
@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [16]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_txt,
)

In [17]:
agent_tool_example.run()


Thought: I need to write the string 'This is a Tool Agent' into a text file named 'tool_agent_example.txt'.

Using Tool: write_str_to_txt

Tool call dict: 
{'name': 'write_str_to_txt', 'arguments': {'string_data': 'This is a Tool Agent', 'txt_filename': 'tool_agent_example.txt'}, 'id': 0}
Data successfully written to tool_agent_example.txt

Tool result: 
None

Observations: {0: None}


"The string 'This is a Tool Agent' has been successfully written to 'tool_agent_example.txt'."

## Defining Agent Dependencies

Let's define two agents now.

In [18]:
agent_1 = Agent(
    name="Poet Agent",
    backstory="You are a well-known poet, who enjoys creating high quality poetry.",
    task_description="Write a poem about the meaning of life",
    task_expected_output="Just output the poem, without any title or introductory sentences",
)

agent_2 = Agent(
    name="Poem Translator Agent",
    backstory="You are an expert translator especially skilled in Ancient Greek",
    task_description="Translate a poem into Ancient Greek", 
    task_expected_output="Just output the translated poem and nothing else"
)

We can define the agent dependencies using the `>>` operator.

In [19]:
agent_1 >> agent_2

Poem Translator Agent

This means `agent_2` depends on `agent_1`. We can check the dependencies and dependents of both agents.

In [20]:
print("Agent 1 dependencies: ", agent_1.dependencies)
print("Agent 1 dependents: ", agent_1.dependents)
print("Agent 2 dependencies: ", agent_2.dependencies)
print("Agent 2 dependents: ", agent_2.dependents)

Agent 1 dependencies:  []
Agent 1 dependents:  [Poem Translator Agent]
Agent 2 dependencies:  [Poet Agent]
Agent 2 dependents:  []


Now, if we run `agent_1`, the results will be added to `agent_2`'s context.

In [21]:
print(agent_1.run())

In the cradle of dawn, where whispers reside,  
Life unfurls its tapestry, vast and wide.  
In the dance of shadows and the play of light,  
We seek the meaning, both day and night.  

In the laughter of children, pure and bright,  
In the tears of the weary, in the silent night,  
In the rustle of leaves and the ocean's roar,  
Life's meaning hides in the depths of lore.  

In the bonds we form, in love's gentle embrace,  
In the trials we face, in the dreams we chase,  
In the moments of stillness, in the rush of time,  
Life's essence echoes in a silent rhyme.  

In the quest for truth, in the heart's deep quest,  
In the courage to rise, in the soul's unrest,  
In the beauty of now, in the hope of tomorrow,  
Life's meaning weaves through joy and sorrow.  

In the stars above, in the earth below,  
In the mysteries that we may never know,  
Life's meaning is found in the journey we take,  
In the love we give, and the bonds we make.  


In [22]:
print(agent_2.context)

Poem Translator Agent received context: 
In the cradle of dawn, where whispers reside,  
Life unfurls its tapestry, vast and wide.  
In the dance of shadows and the play of light,  
We seek the meaning, both day and night.  

In the laughter of children, pure and bright,  
In the tears of the weary, in the silent night,  
In the rustle of leaves and the ocean's roar,  
Life's meaning hides in the depths of lore.  

In the bonds we form, in love's gentle embrace,  
In the trials we face, in the dreams we chase,  
In the moments of stillness, in the rush of time,  
Life's essence echoes in a silent rhyme.  

In the quest for truth, in the heart's deep quest,  
In the courage to rise, in the soul's unrest,  
In the beauty of now, in the hope of tomorrow,  
Life's meaning weaves through joy and sorrow.  

In the stars above, in the earth below,  
In the mysteries that we may never know,  
Life's meaning is found in the journey we take,  
In the love we give, and the bonds we make.  


Now, if we run the second agent, it will use the context received from the previous agent to generate its output.

In [23]:
print(agent_2.run())

Ἐν τῇ λίκνῃ τῆς ἕω, ἔνθα ψιθυρισμοὶ κατοικοῦσιν,  
Ζωὴ ἀναπτύσσει τὸν ἱστόν, πλατύς τε καὶ εὐρύς.  
Ἐν τῷ χορῷ τῶν σκιῶν καὶ τῇ παίγνῃ τοῦ φωτός,  
Ζητοῦμεν τὸ νόημα, ἡμέρας καὶ νυκτός.  

Ἐν τῷ γέλωτι τῶν παίδων, καθαρῷ τε καὶ λαμπρῷ,  
Ἐν τοῖς δάκρυσι τῶν κοπιώντων, ἐν τῇ σιγῇ τῆς νυκτός,  
Ἐν τῇ θροΐσσει τῶν φύλλων καὶ τῷ βρυχηθμῷ τῆς θαλάσσης,  
Τὸ νόημα τῆς ζωῆς κρύπτεται ἐν τοῖς βάθεσι τῶν μύθων.  

Ἐν τοῖς δεσμοῖς οὓς συνάπτομεν, ἐν τῇ ἁπαλῇ ἀγκαλίᾳ τῆς ἀγάπης,  
Ἐν ταῖς δοκιμασίαις ἃς ἀντιμετωπίζομεν, ἐν τοῖς ὀνείροις ἃς διώκομεν,  
Ἐν ταῖς στιγμαῖς τῆς ἡσυχίας, ἐν τῷ ῥύμῳ τοῦ χρόνου,  
Ἡ οὐσία τῆς ζωῆς ἀντηχεῖ ἐν ἀφώνῳ ῥυθμῷ.  

Ἐν τῇ ζητήσει τῆς ἀληθείας, ἐν τῇ βαθείᾳ ζητήσει τῆς καρδίας,  
Ἐν τῷ θάρρει ἀναστῆναι, ἐν τῇ ταραχῇ τῆς ψυχῆς,  
Ἐν τῇ καλλονῇ τοῦ νῦν, ἐν τῇ ἐλπίδι τοῦ αὔριον,  
Τὸ νόημα τῆς ζωῆς ὑφαίνεται διὰ χαρᾶς καὶ λύπης.  

Ἐν τοῖς ἄστροις ἄνω, ἐν τῇ γῇ κάτω,  
Ἐν τοῖς μυστηρίοις ἃ οὐκ ἴσως γνῶμεν ποτέ,  
Τὸ νόημα τῆς ζωῆς εὑρίσκεται ἐν τῇ ὁδῷ ἣν πορευόμεθα,  

## The Crew

In [24]:
from src.multi_agent.crew import Crew

In [25]:
with Crew() as crew:
    agent_1 = Agent(
        name="Poet Agent",
        backstory="You are a well-known poet, who enjoys creating high quality poetry.",
        task_description="Write a poem about the meaning of life",
        task_expected_output="Just output the poem, without any title or introductory sentences",
    )

    agent_2 = Agent(
        name="Poem Translator Agent",
        backstory="You are an expert translator especially skilled in Spanish",
        task_description="Translate a poem into Spanish", 
        task_expected_output="Just output the translated poem and nothing else"
    )

    agent_3 = Agent(
        name="Writer Agent",
        backstory="You are an expert transcriber, that loves writing poems into txt files",
        task_description="You'll receive a Spanish poem in your context. You need to write the poem into './poem.txt' file",
        task_expected_output="A txt file containing the greek poem received from the context",
        tools=write_str_to_txt,
    )

    agent_1 >> agent_2 >> agent_3

In [26]:
crew.run()


RUNNING AGENT: Poet Agent

In the quiet whispers of the dawn,  
Where shadows dance and dreams are drawn,  
Life unfurls its tender thread,  
In every step, in words unsaid.  

A tapestry of joy and pain,  
Of sunlit fields and gentle rain,  
In laughter's echo, in a tear,  
The meaning of life becomes clear.  

In moments small, in grand design,  
In hearts that break, in love that binds,  
A fleeting glance, a touch, a sigh,  
The stars that light the velvet sky.  

In seeking truth, in finding grace,  
In every soul, a sacred space,  
Life's meaning shifts, a gentle tide,  
In every heart, where dreams reside.  

In giving more than we receive,  
In holding fast to what we believe,  
In kindness shared, in courage shown,  
The essence of life is truly known.  

In the endless dance of night and day,  
In the silent prayers we softly say,  
Life's meaning, vast as the sea,  
Is found in you, is found in me.  

RUNNING AGENT: Poem Translator Agent

En los susurros tranquilos del aman